In [1]:
#;.pykx.disableJupyter()

In [2]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


# Dictionaries

**Learning Outcomes**

To understand: 
* How to create a dictionary
* How to use a dictionary
* How to amend a dictionary
* How to combine dictionaries

# Introduction
[Dictionaries](https://en.wikipedia.org/wiki/Associative_array) are a key-value data structure that associates a *key* with a *value*. Dictionaries in kdb+/q are created by making an association between two lists of equal length and provide the foundation for creating tables in kdb+. 

## Dictionary decomposition
Dictionaries are a first order datatype in kdb+/q with an associated value of `99h`. 

In [4]:
d: `a`b!1 2  //simple dictionary
d
type d            //type
key d             //keys list 
value d           //value list 
count d           //how many associations we have

a| 1
b| 2
99h
`a`b
1 2
2


Since dictionaries are first order datatypes, we can apply functions to them: 

In [5]:
d
d + 3
d % 1.5  
d * 1 2   //pairwise vector multiplication
d * 1 2 3 //length error

a| 1
b| 2
a| 4
b| 5
a| 0.6666667
b| 1.333333
a| 1
b| 4


QError: length

# Creating a dictionary

The primitive [`!`](https://code.kx.com/q/ref/dict/) (spoken as "bang") is used to create dictionaries - we can simply place a `!` between our list of keys and our list of values:

In [6]:
names:`john`steve`rachel
ages:20 31 22
show dict:names!ages;  // define a dictionary called dict

john  | 20
steve | 31
rachel| 22


Neither the keys nor values need be simple lists, either of them can be general lists - they just need to be the same length. Below are some examples:

In [7]:
(`Arthur`Dent; `Zaphod`Beeblebrox; `Ford`Prefect)! 100 42 150 //using nested symbol lists

1001 1002 1003!(`Arthur`Dent; `Zaphod`Beeblebrox; `Ford`Prefect)

Arthur Dent      | 100
Zaphod Beeblebrox| 42
Ford   Prefect   | 150
1001| Arthur Dent      
1002| Zaphod Beeblebrox
1003| Ford   Prefect   


##### Exercise 

Create a dictionary called `food` which contains the following:
* <code>\`milk\`bread\`biscuits</code> as a key
* <code>1.3 2.0 3.5</code> as a value


In [ ]:
food:`milk`bread`biscuits!1.3 2.0 3.5
food

In [18]:
food:`milk`bread`biscuits!1.3 2.0 3.5
food

milk    | 1.3
bread   | 2
biscuits| 3.5


## Implicit dictionary typing

We can create an empty untyped dictionary using empty key and value lists, or an empty typed dictionary by casting our empty key and value lists: 

In [9]:
show a:()!()          //general dictionary - currently untyped 
type value a
show b:(`$())!"f"$()  //typed empty dictionary - will force conformity to this datatype for any updates
type value b

0h
9h


The untyped dictionary above is interesting - while it currently isn't typed, it will assume a "typing" of the first inserted value. 

If we wanted to create a *true* general dictionary that won't lose it's "untyped" nature we can do so as follows: 

In [10]:
guaranteedUnTyped: enlist[`]!enlist(::)   //we're using enlist here because ! only works with lists
type value guaranteedUnTyped

0h


##### Exercise

Create a single element dictionary called `months`, with one key `January` and a value of 1

In [ ]:
//we don't need the square brackets on the RHS because of operator precendence
show months: enlist[`January]! enlist 1    

In [12]:
months:(enlist `January)!enlist 1
months

January| 1


## Column dictionaries

Column dictionaries are the foundation for tables. These are a special subset of dictionaries with a mapping from a simple list of symbols to a rectangular list of lists.

In [13]:
columnDict: `col1`col2!(1 2 3; 4 5 6)
columnDict

col1| 1 2 3
col2| 4 5 6


# Dictionary Retrieval

Since dictionaries are commonly used for mappings, it's important we understand how we can use them to retrieve the values associated with our keys or vice versa.

## Key lookup 
In most cases dictionaries are accessed by specifying the key(s) we wish to access and the corresponding values are returned.

In [14]:
dict
dict `john
dict[`john]  //functional syntax 
dict[`sarah] //value that isn't in our dictionary- we get a null of the same type as our first dictionary value 

john  | 20
steve | 31
rachel| 22
20
20
0N


We will get a type error if we try to pass keys that do not match the type of our key list: 

In [15]:
dict["igor"]

QError: type

Dictionaries are very useful for mappings because we can pass not just one input, but many input values, effectively mapping our list to the same domain as our dictionary

In [16]:
L:`john`rachel`sarah`rachel
dict[L]      //by providing a list of keys we can retrieve multiple values

20 22 0N 22


##### Exercise 

Using the `food` dictionary that we have defined in the our previous exercise, find the value corresponding with milk.

In [ ]:
food[`milk] //functional syntax 
food`milk 

In [19]:
food[`milk]

1.3


## Reverse Lookup 
We can use the `?` operator to find items in a dictionary- just like we did with lists! For a dictionary this will return the key associated with that value:

In [21]:
10 20 30 40?20 //finds the first index that the value occured at

dict
dict?20 
dict?24        //if the value isn't there, it returns a null of the same type as the first key

1
john  | 20
steve | 31
rachel| 22
john



##### Exercise 

Create a birth month dictionary (`birthMonth`) and create entries for John, Steve and Rachel -> JAN,SEPT,OCT accordingly. Store the keys as symbols, and values as strings. 

Using this dictionary find out:  
* What is Steve's birth month?
* Whose birthday is in October?

In [ ]:
//dictionary definition
show birthMonth: `john`steve`rachel!("JAN";"SEPT";"OCT")

//When is steves birth month?
birthMonth[`steve]

//Whose birth month is October?
birthMonth?"OCT"

In [23]:
birthMonth:`John`Steve`Rachel!("JAN";"SEPT";"OCT")
birthMonth
birthMonth[`Steve]
birthMonth?"OCT"

John  | "JAN"
Steve | "SEPT"
Rachel| "OCT"
SEPT
Rachel


# Amending dictionaries
Now that we know how to create a dictionary, we will discuss how to amend an entry in our dictionary.

## Updating values 
Taking the dictionary `dict` from above, let's say that we need to update some people's ages. We can update a dictionary value by specifying the key and using explicit reassignment.

In [24]:
show dict
dict[`rachel] //extracting the existing value
dict[`rachel]:26        // set a value
dict

22
john  | 20
steve | 31
rachel| 26
john  | 20
steve | 31
rachel| 22


If we want to decrease John's age by 1, we can also do the following:

In [25]:
dict[`john]-:1             // decrement a value 
dict 

dict[`john]:dict[`john]-1   
dict 

john  | 19
steve | 31
rachel| 26
john  | 18
steve | 31
rachel| 26


##### Exercise 

Using our dictionary `birthMonth` from previous questions, update Steve's birth month to the shorthand "AUG" for August.

In [ ]:
//dictionary definition
birthMonth:`john`steve`rachel!("JAN";"SEPT";"OCT");
//reassigning the value for `steve
birthMonth[`steve]:"AUG";
//updated dictionary
birthMonth

In [28]:
birthMonth[`Steve]: "AUG"

## Removing entries
We can use [`_`](https://code.kx.com/q/ref/drop/#drop-keys-from-a-dictionary)(referred to as [drop](https://code.kx.com/q/ref/drop/#drop-keys-from-a-dictionary)) to remove key-value pairs from a dictionary. 

In [29]:
//with a list
2 _ til 10   //using _ with a list to remove the first 2 items
//creating a dictionary
show dict: `tom`brian`steve`sarah`jane`joanne!(18 19 20 31 21 27)

2 3 4 5 6 7 8 9
tom   | 18
brian | 19
steve | 20
sarah | 31
jane  | 21
joanne| 27


If we want to remove Tom and Jane from our dictionary we can do the below:

In [30]:
`tom`jane _ dict           // whitespace is required to the left of _
`joanne _ dict             // remove one entry
`mark`sarah _ dict         // removing a key that does not exist has no effect & won't error
key[dict] _ dict           // returns an empty dictionary

brian | 19
steve | 20
sarah | 31
joanne| 27
tom  | 18
brian| 19
steve| 20
sarah| 31
jane | 21
tom   | 18
brian | 19
steve | 20
jane  | 21
joanne| 27
(`symbol$())!`long$()


##### Exercise 

Return the `birthMonth` dictionary without the entry for John:

In [ ]:
//dictionary definition
show birthMonth 

//returning without John's key value
`john _ birthMonth

In [32]:
`John _ birthMonth

Steve | "AUG"
Rachel| "OCT"


## Subsetting entries 
We can use the primitive `#`(referred to as "take") to explicitly choose only certain values in our dictionary: 

In [34]:
`tom`jane # dict            //selecting just the values for Tom and Jane
`timothy`sarah # dict       //selecting values that aren't present assigns a null value to that key
23^`timothy`sarah # dict    //setting a default of 23 for any missing values

tom | 18
jane| 21
timothy| 
sarah  | 31
timothy| 23
sarah  | 31
`steve`sarah


## Appending to a dictionary
Adding a new value to a dictionary is simple and is done in the same way as amending a dictionary:


In [ ]:
dict

//adding one new value
dict[`tim]:35;           //the assignment method
dict

//can add in bulk
dict[`sally`joe]: 23 52  //assignment is pairwise between the keys and the supplied vector
dict

##### Exercise

Add Tom and Brian's birth months (FEB,DEC) to our `birthMonth` dictionary - this time add them as symbols.

In [ ]:
birthMonth                //our starting dictionary
type value birthMonth     //the implicit dictionary type is general 

//assigning new values as symbols
birthMonth[`tom`brian]:`FEB`DEC

birthMonth

The reason the above does not throw an error when in other cases it would is because list of string lists are general lists so the dictionary will accept any type input. 

In [35]:
birthMonth[`Tom`Brian]:`FEB`DEC
birthMonth

John  | "JAN"
Steve | "AUG"
Rachel| "OCT"
Tom   | `FEB
Brian | `DEC


# Combining dictionaries
We will often have to work with and manipulate multiple dictionaries and it is important to understand how to combine them, either through operations like addition, or joining.

## Joining two dictionaries

The join operator `,` can be used to merge two dictionaries, let's create two dictionaries with uncommon keys and merge them:

In [36]:
d1:`AAPL`IBM`KX!10 20 30
d1

d2:`FD`MSFT!40 50
d2

d1,d2               //Using the join operator 

AAPL| 10
IBM | 20
KX  | 30
FD  | 40
MSFT| 50
AAPL| 10
IBM | 20
KX  | 30
FD  | 40
MSFT| 50


The join operator `,` uses upsert semantics meaning if the key exists it will update the dictionary with the corresponding value. If the value does not exist, it will insert a new key value pair.  Let's look at what happens when we join two dictionaries with common keys.

In [37]:
//new dictionaries
d1:`AAPL`IBM`KX!10 20 30
d2:`KX`FD`MRP!100 200 300    

key[d1] inter key[d2]        //common key Kx

,`KX


In [38]:
d1,d2                        //value in d2 prevails
d2,d1                        //value in d1 prevails
(d2,d1)~d1,d2

AAPL| 10
IBM | 20
KX  | 100
FD  | 200
MRP | 300
KX  | 30
FD  | 200
MRP | 300
AAPL| 10
IBM | 20
0b


##### Exercise 

Using the dictionaries, <code>d1:\`a\`b\`c!1 2 3</code> and <code>d2:\`a\`b\`d!2 2 5</code>

Create a dictionary `d3` by combining `d1` and `d2` - we want the values in `d2` to take priority.

In [ ]:
d1:`a`b`c!1 2 3
d2:`a`b`d!2 2 5

d3:d1,d2
d3

In [40]:
d1:`a`b`c!1 2 3
d2:`a`b`d!2 2 5
d3:d1,d2
d3

a| 2
b| 2
c| 3
d| 5


## Coalesce ^

The Coalesce operator is similar to `,` in that it employs upsert semantics to merge two dictionaries.

In [42]:
d1:`a`b`c!10 20 30
d1

d2:`b`c`d!200 0N 400
d2

"colaesce"
d1^d2   //rather than keeping the null value of c in d2, the value from d1 is retained

a| 10
b| 20
c| 30
b| 200
c| 
d| 400
colaesce
a| 10
b| 200
c| 30
d| 400


##### Exercise 

Given our previous dictionary `d3`(<code>d3:\`a\`b\`c\`d!2 2 3 5</code>) and a new dictionary <code>d4:\`c\`d\`e\`f!20 0N 31 5</code>, use these to create the following dictionary: 

A.

    a| 2 
    b| 2 
    c| 20 
    d| 5 
    e| 31 
    f| 5  

In [ ]:
d3:`a`b`c`d!2 2 3 5
d4:`c`d`e`f!20 0N 31 5
//The resultant dictionary has the same values as the values from d4, except where d4 values are null. 
//The resultant dictionary has been filled with the d3 value
d3^d4

In [44]:
d4:`c`d`e`f!20 0N 31 5
d3^d4

a| 2
b| 2
c| 20
d| 5
e| 31
f| 5


## Mathematical Operations

Following the general pattern that we have seen up until now, arithmetic operators are executed on common keys. If uncommon keys exists, they will remain unchanged.

In [45]:
d1:`IBM`FD`KX!10 20 30
d1

d2:`FD`KX`MRP!200 300 400
d2 

d1+d2 

IBM| 10
FD | 20
KX | 30
FD | 200
KX | 300
MRP| 400
IBM| 10
FD | 220
KX | 330
MRP| 400


##### Exercise 

Using the previously defined dictionaries `d3` and `d4`, use them to create the below dictionary. 
    
    c| 6.666667 
    d| 0n
    e| 31 
    f| 5 
    a| 2 
    b| 2 

In [ ]:
d3:`a`b`c`d!2 2 3 5
d4:`c`d`e`f!20 0N 31 5


//the dictionary has been unaffected in uncommon keys, for common keys c and d they have been divided
    //our first "hint" it's division is that our type has changed - 6.666667 and 0n show us that
d4%d3

In [47]:
d4 % d3

c| 6.666667
d| 0n
e| 31
f| 5
a| 2
b| 2
